In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from eo_tools.util import show_cog
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# import numpy as np
# from folium import LayerControl

In [ ]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230903T183344_20230903T183412_050167_0609B4_100E.SAFE"
secondary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230915T183345_20230915T183413_050342_060F9F_85A4.SAFE"
# out_dir = f"/data/res/test_new_proc"
out_dir = f"/data/res/test_param_valid"
iw = 2
pol = "vv"
min_burst = 1
max_burst = None

In [ ]:
from eo_tools.S1.process import preprocess_insar_iw

preprocess_insar_iw(
    primary_dir,
    secondary_dir,
    out_dir,
    iw=iw,
    pol=pol,
    min_burst=min_burst,
    max_burst=max_burst,
)

In [ ]:
from eo_tools.S1.process import slc2geo, interferogram, coherence, amplitude
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_ifg = f"{out_dir}/ifg.tif"
file_amp = f"{out_dir}/amp.tif"
file_coh = f"{out_dir}/coh.tif"
file_phi_geo = f"{out_dir}/phi_geo.tif"
file_amp_geo = f"{out_dir}/amp_geo.tif"
file_coh_geo = f"{out_dir}/coh_geo.tif"
file_lut = f"{out_dir}/lut.tif"
# computing outputs in the radar geometry
interferogram(file_prm, file_sec, file_ifg)
coherence(file_prm, file_sec, file_coh, box_size=5, magnitude=True)
amplitude(file_prm, file_amp)

# multilooking and geocoding
slc2geo(file_ifg, file_lut, file_phi_geo, 2, 8, 3, True)
slc2geo(file_coh, file_lut, file_coh_geo, 2, 8, 3, False)
slc2geo(file_amp, file_lut, file_amp_geo, 2, 8, 3, False, True)

In [ ]:
# import folium
# m = Map()
# tile = folium.TileLayer(
#         tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#         attr = 'Esri',
#         name = 'Esri Satellite',
#         overlay = False,
#         control = True
#        ).add_to(m)

# # hack: avoid titiler caching the tiles
# scale_phi = np.pi + np.random.rand() * 1e-6
# rnd_coh = np.random.rand() * 1e-6

# show_cog("/data/res/test_merge_coh2.tif", m,rescale=f"0,{1+rnd_coh}")
# show_cog("/data/res/test_merge_phi2.tif", m,rescale=f"-{scale_phi}, {scale_phi}",colormap=palette_phi())
# LayerControl().add_to(m)
# m